In [2]:
import pandas as pd
from google.cloud import storage
!pip install gcsfs --quiet

# carregamento do dataset consolidado
gcs_path = "gs://projeto-recomendador/historico_compras_clientes.csv"  

df = pd.read_csv(gcs_path)

print(f"Shape do dataset: {df.shape}")
df.head()


Shape do dataset: (111047, 6)


,customer_id,product_id,order_purchase_timestamp,product_category_name,customer_city,customer_state
0,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,2017-10-02 10:56:33,utilidades_domesticas,sao paulo,SP
1,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,2018-07-24 20:41:37,perfumaria,barreiras,BA
2,41ce2a54c0b03bf3443c3d931a367089,aa4383b373c6aca5d8797843e5594415,2018-08-08 08:38:49,automotivo,vianopolis,GO
3,f88197465ea7920adcdbec7375364d82,d0b61bfb1de832b15ba9d266ca96e5b0,2017-11-18 19:28:06,pet_shop,sao goncalo do amarante,RN
4,8ab97904e6daea8866dbdbc4fb7aad2c,65266b2da20d04dbe00c5c2d3bb7859e,2018-02-13 21:18:39,papelaria,santo andre,SP


In [3]:
# Verificações

# tipos de dados
print("\nTipos de dados:")
print(df.dtypes)

# colunas com valores únicos
print("\nNº de valores únicos por coluna:")
print(df.nunique())

# valores nulos (confirmação final)
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# estatísticas básicas para variáveis numéricas
print("\nEstatísticas descritivas:")
print(df.describe())

# Verificações colunas categóricas
print("\nCategorias distintas em colunas de texto:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].nunique()} categorias")



Tipos de dados:
customer_id                 object
product_id                  object
order_purchase_timestamp    object
product_category_name       object
customer_city               object
customer_state              object
dtype: object

Nº de valores únicos por coluna:
customer_id                 97277
product_id                  32341
order_purchase_timestamp    96742
product_category_name          73
customer_city                4095
customer_state                 27
dtype: int64

Valores nulos por coluna:
customer_id                 0
product_id                  0
order_purchase_timestamp    0
product_category_name       0
customer_city               0
customer_state              0
dtype: int64

Estatísticas descritivas:
                             customer_id                        product_id  \
count                             111047                            111047   
unique                             97277                             32341   
top     fc3d1daec319d62d49b

In [4]:
# Conveção timestamp para datetime
df['order_purchase_timestamp'] = pd.to_datetime(
    df['order_purchase_timestamp'], errors='coerce'
)

# Converção colunas categóricas, para performance
categorical_cols = ['product_category_name', 'customer_city', 'customer_state']
for col in categorical_cols:
    df[col] = df[col].astype('category')
    
print(df.dtypes)

customer_id                         object
product_id                          object
order_purchase_timestamp    datetime64[ns]
product_category_name             category
customer_city                     category
customer_state                    category
dtype: object


In [5]:
# Objetivo: gerar variáveis agregadas para traçar perfis de clientes de acordo com o comportamento de compras

# considerar apenas combinações únicas de cliente + data da compra (1 linha por pedido real)
df_pedidos_unicos = df.drop_duplicates(subset=['customer_id', 'order_purchase_timestamp'])

# recência: última compra até o presente
data_atual = df_pedidos_unicos['order_purchase_timestamp'].max()

# Há clientes com mais de um pedido feito em datas diferentes, no dataset?
# número de dias distintos com compras por cliente
compras_por_cliente = df.groupby('customer_id')['order_purchase_timestamp'].nunique()

# quantos clientes têm mais de uma compra em dias diferentes
compras_por_cliente.value_counts().sort_index()



order_purchase_timestamp
1    97277
Name: count, dtype: int64

In [13]:
# como não há mais de um pedido por cliente, a análise não poderá ser feita em cima de histórico de múltiplas compras.
# features por cliente
df_agg = df_pedidos_unicos.groupby('customer_id').agg(
    categoria_mais_frequente=('product_category_name', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    estado_cliente=('customer_state', 'first'),
    cidade_cliente=('customer_city', 'first'),
    data_compra=('order_purchase_timestamp', 'min')
)

# recência da última compra
df_agg['recencia_compra'] = (data_atual - df_agg['data_compra']).dt.days

# exclusão das colunas de data, que não agregam ao modelo
df_agg.drop(['data_compra'], axis=1, inplace=True)

df_agg.reset_index(inplace=True)
df_agg.describe()
# df_agg.head()

,recencia_compra
count,97277.000000
mean,244.072597
std,153.411569
min,0.000000
25%,120.000000
50%,225.000000
75%,354.000000
max,728.000000


In [14]:
# recuperar o último product_id por cliente
produto_por_cliente = df_pedidos_unicos.sort_values('order_purchase_timestamp').groupby('customer_id').last()[['product_id']]
df_modelo = df_agg.merge(produto_por_cliente, on='customer_id', how='left')

df_modelo.head()


,customer_id,categoria_mais_frequente,estado_cliente,cidade_cliente,recencia_compra,product_id
0,00012a2ce6f8dcda20d059ce98491703,brinquedos,SP,osasco,292,64315bd8c0c47303179dd2e25b579d00
1,000161a058600d5901f007fab4c27140,beleza_saude,MG,itapecerica,413,84183944dc7cddca87a5d384452c1d3c
2,0001fd6190edaaf884bcaf3d49edf079,bebes,ES,nova venecia,551,9df2b21ec85378d71df4404712e17478
3,0002414f95344307404f0ace7a26f1d5,cool_stuff,MG,mendonca,382,af3ec22cce878225aae6d9eb6c7a78eb
4,000379cdec625522490c315e70c7a9fb,cama_mesa_banho,SP,sao paulo,153,868b3136c5b206f91b8208fbfdf2cb7c


In [15]:
# salvar CSV local
local_file = "base_modelo.csv"
df_modelo.to_csv(local_file, index=False)

# upload para o bucket
destination_blob = "base_modelo.csv"
bucket_name = 'projeto-recomendador'

client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob)
blob.upload_from_filename(local_file)

print(f"Upload concluído: gs://{bucket_name}/{destination_blob}")

Upload concluído: gs://projeto-recomendador/base_modelo.csv


In [16]:
# ranking de produtos por categoria
# Contagem de produtos por categoria
ranking_produtos = df.groupby(['product_category_name', 'product_id']) \
                     .size() \
                     .reset_index(name='frequencia') \
                     .sort_values(['product_category_name', 'frequencia'], ascending=[True, False])

# Adiciona ranking por categoria
ranking_produtos['rank'] = ranking_produtos.groupby('product_category_name')['frequencia'] \
                                            .rank(method='first', ascending=False)

# Dicionário: categoria -> lista de produtos por ordem de popularidade
dict_produtos_por_categoria = ranking_produtos.groupby('product_category_name')['product_id'] \
                                              .apply(list) \
                                              .to_dict()



/var/tmp/ipykernel_3675/1181894731.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ranking_produtos = df.groupby(['product_category_name', 'product_id']) \
/var/tmp/ipykernel_3675/1181894731.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ranking_produtos['rank'] = ranking_produtos.groupby('product_category_name')['frequencia'] \
/var/tmp/ipykernel_3675/1181894731.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning

In [17]:
# Top 5 em beleza_saude (exemplo)
dict_produtos_por_categoria.get('beleza_saude')[:5]


['154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318',
 '7c1bd920dbdf22470b68bde975dd3ccf',
 'bb50f2e236e5eea0100680137654686c',
 '19c91ef95d509ea33eda93495c4d3481']

In [18]:
import json

# Salva localmente como JSON
with open('ranking_produtos.json', 'w') as f:
    json.dump(dict_produtos_por_categoria, f)

# Faz upload para o GCS
bucket_name = 'projeto-recomendador'
destination_blob_name = 'ranking_produtos.json'
source_file_name = 'ranking_produtos.json'

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)
